In [52]:
# Spacy code
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# My code
import pandas as pd
from ast import literal_eval

# Directories
data_dir = Path('./data')
output_dir = Path('./output')

In [119]:
ner_data = pd.read_excel(data_dir / 'headline_entities_corrected_v0.xlsx', 1, dtype = {'entities' : str, 'headline' : str}).replace('\xa0', ' ', regex = True)

ner_data['entities'] = ner_data.entities.replace('  +', ' ', regex = True).replace(['\’', '\‘'], "'", regex = True).replace(", [^'\(]", ", '", regex = True).apply(literal_eval)

In [120]:
ner_vals = ner_data.values

In [122]:
# Bit of a bug when have accents
formatted_data = [(headline, {'entities' : [(headline.find(ent[0]), headline.find(ent[0]) + len(ent[0]) - 1, ent[1]) for ent in entities]}) for entities, headline in ner_vals]

In [134]:
def train_ner(data, new_model_name, new_labels = None, model=None, output_dir=None, n_iter=30):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")
    
    if new_labels:
        for new_label in new_labels:
            ner.add_label(new_label)  # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(data)
            batches = minibatch(data, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "Gareth Southgate keeps on changing England for the better after testing night in Montenegro"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
    
    if len(doc.ents) == 0:
        print('None found')

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [130]:
new_labels = ['TEAM', 'PERSON', 'COMPETITION']

In [135]:
train_ner(formatted_data, 'football-ner', new_labels = new_labels, model=None, output_dir=output_dir / 'football-ner', n_iter=3)

Created blank 'en' model
Losses {'ner': 64.50968173131561}
Losses {'ner': 4.758178282336072e-17}
Losses {'ner': 3.724533452799473e-18}
Entities in 'Gareth Southgate keeps on changing England for the better after testing night in Montenegro'
None found
Saved model to output/football-ner
Loading from output/football-ner
